该文件主要是为了生成需要的数据，并将数据保存到 data.db 中

In [58]:
import os
import string
import datetime
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [20]:
# 数据库地址：数据库放在上一级目录下
db_path = os.path.join(os.path.dirname(os.getcwd()),"data.db")
engine_path = "sqlite:///" + db_path
# 创建数据库引擎
engine = create_engine(engine_path)

### 用户数据
> 保存到数据库 users 表中

In [74]:
name_str_list = [i for i in string.ascii_letters+string.digits]

In [84]:
# 随机生成10000个用户的信息
user_df = pd.DataFrame(data={
    "user_id":range(1,10001),
    "username":["".join(np.random.choice(name_str_list,6)) for _ in range(10000)],
    "age":np.random.choice(range(20,50),10000),
})

In [85]:
user_df.sample(2)

,user_id,username,age
7468,7469,mQ2qGm,37
234,235,HkIhIx,35


In [86]:
user_df.to_sql("users", engine, if_exists='replace', index=False)

### 商品评价数据
> 商品评论数据来自 [阿里云天池比赛](https://tianchi.aliyun.com/competition/entrance/531890/introduction)
>
> 数据保存到数据库 comment 表中

In [87]:
comment_df = pd.read_csv("./earphone_sentiment.csv")

In [88]:
comment_df.sample(2)

,content_id,content,subject,sentiment_word,sentiment_value
14255,14284,说句冒犯的话，怎么感觉是在狂吹，然后找下家出货,其他,NaN,0
174,180,到家翻出吃灰已久的240DF，细细听来还是真是不错。以前觉得不出彩似乎是我的音量开得太保守了。,其他,不错,1


In [89]:
comment_df.shape

(17176, 5)

In [91]:
# 添加创建时间
index = pd.date_range("2021-01-01", "2021-12-31")

# 随机选择 17176 条时间
comment_df["create_time"] = np.random.choice(a=index,size=17176)

# 添加用户id
comment_df["user_id"] = range(1,len(comment_df)+1)

In [92]:
comment_df.head(2)

,content_id,content,subject,sentiment_word,sentiment_value,create_time,user_id
0,0,Silent Angel期待您的光临，共赏美好的声音！,其他,好,1,2021-10-14,1
1,2,这只HD650在1k的失真左声道是右声道的6倍左右，也超出官方规格参数范围（0.05%），看...,其他,NaN,0,2021-07-16,2


In [93]:
comment_df.to_sql("comment", engine, if_exists='replace', index=False)

### 观研报告网数据
> 观研报告网：http://data.chinabaogao.com/ 
>
> read_html() 官网地址：[点击跳转](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html?highlight=read_html)
>
> 网页中有表格元素（tbody）均可以下载  


In [9]:
# 举例说明，该 url 下有两个表格
# 其他页面数据，只需修改该 url 
url = "http://data.chinabaogao.com/dianxin/2021/11305620U2021.html"

result = pd.read_html(io=url,header=0)

for d in result:
    display(d)